## Name: Roland Vu

## Title: Graph Analysis for Maximum Chromatic Vectors

### Background

In this project, I have been exploring graph coloring and the chromatic polynomial as part of my undergraduate mathematics research. I analyzed a given dataset of chromatic polynomial of all connected graphs up to order 9, provided by my research advisor, and studied their respective chromatic vectors. I also investigated various graph properties to gain insights into the characteristics of graphs that yield the largest chromatic vectors.

### Python Packages and Methods

For this research project, I used the following Python packages:

- Pandas: for data manipulation and analysis.
- NetworkX: for graph theory and graph operations.
- NumPy: for data manipulation and matrix operations.
- SciPy: for statistical computations.
- Matplotlib: for graph visualization.

### Code Outline

The code for my research project will accomplish the following tasks:

1. Load and preprocess the dataset of connected graphs up to order 9 using Pandas.
2. Sort the dataset by the chromatic vector and group it by vertices and edges using Pandas.
3. Utilize NetworkX in conjunction with Pandas to calculate and add the following graph properties to the dataset:
   - Number of spanning tree
   - Number of biconnected components
   - Number of 3-cycles
   - Number of 4-cycles
   - Number of 5-cycles
   - Number of 6-cycles
   - Number of spanning trees
   - Degree sequence
   - Degree variance
   - Degree deviation
4. Utilize Pandas to analyze the dataset and identify the graphs that yield the largest chromatic vectors in their respective groups.
5. Statistical analysis using Pandas and SciPy to calculate the percentile rank of maximum chromatic vector graph properties versus other non-max graphs within their group.
6. Visualize graphs and results using NetworkX in conjunction with Matplotlib for data visualization.

In [1]:
# Import pandas, networkx, and matplotlib and some helper functions
import numpy as np
import pandas as pd
import networkx as nx
from scipy import stats
import matplotlib.pyplot as plt
from ast import literal_eval as make_tuple

In [ ]:
# Read original dataset csv into DataFrame
df = pd.read_csv('dataset/vector9.csv')

# Remove first couple rows of graph data that is below 6 vertices because they are too small to be useful
df = df.drop(df.index[0:29])

# Coalesce all chromatic coefficients into a tuple and drop unused columns
df['Vector'] = list(zip(df.Coef1, df.Coef2, df.Coef3, df.Coef4, df.Coef5, df.Coef6, df.Coef7, df.Coef8, df.Coef9))
df = df.drop(['Coef1', 'Coef2', 'Coef3', 'Coef4', 'Coef5', 'Coef6', 'Coef7', 'Coef8', 'Coef9'], axis=1)

# Display dataframe
df

In [ ]:
# Stable sort means preserving order if elements are equal
# Sorting by tuple representation of the chromatic vector as defined in Python standard library
df_sorted = df.sort_values(['Vector'], kind='stable')

# After that, stable sort DataFrame
df_sorted = df_sorted.sort_values(['Vertices', 'Edges'], kind='stable')

# Display dataframe
df_sorted

In [ ]:
# This cell compute the number of biconnection, 3-cycle, 4-cycle, 5-cycle, 6-cycle, and spanning tree for all graphs
# It will also compute the degree sequence, degree variation, and degree deviation for all graphs
# This will take at least 2-3 hours to complete on a powerful machine, so save your result after the first time

# Instantiate list to hold results
spanning_tree_list = list()
biconnected_list   = list()
three_cycle_list   = list()
four_cycle_list    = list()
five_cycle_list    = list()
six_cycle_list     = list()
degree_list        = list()
variance_list      = list()
deviation_list     = list()

# For each graphs, do:
for graph6id in df_sorted['G6id']:
    
    # Convert graph6id to instance of NetworkX graph
    G = nx.from_graph6_bytes(graph6id.encode("ascii"))

    # Get the Laplacian matrix of G as a numpy array
    matrix = nx.laplacian_matrix(G).toarray()

    # Delete a row and a column from matrix
    matrix = np.delete(matrix, 0, axis=0)
    matrix = np.delete(matrix, 0, axis=1)

    # Append the determinant of submatrix, aka number of spanning trees, to list
    spanning_tree_list.append(int(round(np.linalg.det(matrix))))

    # Get number of biconnection and append to list
    biconnected_list.append(len(list(nx.biconnected_components(G))))
    
    # Generate all simple cycles for graph G
    cycle_list = list(nx.simple_cycles(G.to_directed()))
    
    # Trim all '2-cycle' and sort list
    cycle_list = [sorted(x) for x in cycle_list if len(x) >= 3]
    
    # Append number of n-cycle to n_list
    three_cycle_list.append(len(set(map(tuple, [x for x in cycle_list if len(x) == 3]))))
    four_cycle_list.append(len(set(map(tuple, [x for x in cycle_list if len(x) == 4]))))
    five_cycle_list.append(len(set(map(tuple, [x for x in cycle_list if len(x) == 5]))))
    six_cycle_list.append(len(set(map(tuple, [x for x in cycle_list if len(x) == 6]))))

    # Get degree iterator from graph and map into a tuple
    degree = tuple(map(lambda x: x[1], G.degree))

    # Append degree tuple to list
    degree_list.append(degree)

    # Compute the degree mean
    mean = sum(degree) / len(degree)

    # Compute the variance
    variance = sum(map(lambda x: (x - mean) ** 2, degree)) / len(degree)

    # Append variance rounded to 4 decimals
    variance_list.append(round(variance, 4))

    # Compute and append degree standard deviation rounded to 4 decimals
    deviation_list.append(round(variance ** 0.5, 4))

# Add columns to dataframe
df_sorted['SpanningTree'] = spanning_tree_list
df_sorted['Biconnection'] = biconnected_list
df_sorted['3-cycle']      = three_cycle_list
df_sorted['4-cycle']      = four_cycle_list
df_sorted['5-cycle']      = five_cycle_list
df_sorted['6-cycle']      = six_cycle_list
df_sorted['Degree']       = degree_list
df_sorted['Variance']     = variance_list
df_sorted['Deviation']    = deviation_list

# Display dataframe
df_sorted

In [8]:
# This function determines if two chromatic vectors are incomparable or not, it is used to find the maximum element in the partial ordered set
# Returns False if two chromatic vector are comparable, True if incomparable
def incomparable(v1, v2):
    greater = [True if a >= b else False for a, b in zip(v1, v2)]
    lesser  = [True if a <= b else False for a, b in zip(v1, v2)]
    return not all(greater) != all(lesser)


# Create new column for indicating maximum element within its group, default to False
df_sorted = df_sorted.assign(Maximum=False)

maximum_list = list()

# Group by vertices and edges, and for each group dataframe:
groups = df_sorted.groupby(['Vertices', 'Edges'])
for name, group in groups:

    # Get the maximum of current group, which is always the last element
    maximum = group.iloc[-1]
    # Apply the comparison function to all vectors to the maximum in the group, and extend result to end of list
    maximum_list.extend(list(group['Vector'].transform(lambda x: incomparable(make_tuple(maximum['Vector']), make_tuple(x)))))

# Set result to column
df_sorted['Maximum'] = maximum_list

# Display dataframe
display(df_sorted)

,G6id,Vertices,Edges,Vector,Biconnection,3-cycle,4-cycle,5-cycle,6-cycle,Degree,Variance,Deviation,SpanningTree,Maximum
Index,,,,,,,,,,,,,,
29,E?Bw,6,5,"(1, 5, 10, 10, 5, 1, 0, 0, 0)",5,0,0,0,0,"(1, 1, 1, 1, 1, 5)",2.2222,1.4907,1,True
30,E?bo,6,5,"(1, 5, 10, 10, 5, 1, 0, 0, 0)",5,0,0,0,0,"(2, 1, 1, 1, 1, 4)",1.2222,1.1055,1,True
32,E?qo,6,5,"(1, 5, 10, 10, 5, 1, 0, 0, 0)",5,0,0,0,0,"(2, 1, 1, 1, 2, 3)",0.5556,0.7454,1,True
33,E?ow,6,5,"(1, 5, 10, 10, 5, 1, 0, 0, 0)",5,0,0,0,0,"(1, 1, 1, 1, 3, 3)",0.8889,0.9428,1,True
43,ECR_,6,5,"(1, 5, 10, 10, 5, 1, 0, 0, 0)",5,0,0,0,0,"(2, 2, 1, 1, 1, 3)",0.5556,0.7454,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273187,H]~v~~~,9,33,"(1, 33, 465, 3632, 17079, 49097, 83375, 75398,...",1,63,126,126,84,"(7, 7, 7, 7, 7, 7, 8, 8, 8)",0.2222,0.4714,2250423,True
273165,HV~~~~~,9,34,"(1, 34, 490, 3892, 18529, 53746, 91860, 83448,...",1,71,120,126,84,"(7, 6, 7, 8, 8, 8, 8, 8, 8)",0.4691,0.6849,2834352,False
273188,H]~~~~~,9,34,"(1, 34, 491, 3913, 18704, 54481, 93484, 85212,...",1,70,126,126,84,"(7, 7, 7, 7, 8, 8, 8, 8, 8)",0.2469,0.4969,2893401,True


In [11]:
# Output (or overwrite) to new csv
df_sorted.to_csv('dataset/sorted.csv')

# Output (or overwrite) to new csv
maximum_df = df_sorted[df_sorted['Maximum'] == True]
maximum_df.to_csv('dataset/maximum.csv')

# Group by vertices and edges, and for each group dataframe:
groups = df_sorted.groupby(['Vertices', 'Edges'])

for name, group in groups:

    # Save the group into its own CSV file, for easier analytics
    group.to_csv(f'dataset/group/{name[0]}_{name[1]}.csv')

In [2]:
# This loads in the intermediate CSV with result saved halfway through so that no expensive computation needs to be recalculated
df_sorted = pd.read_csv('dataset/sorted.csv', index_col=0)

# Initialize new empty dataframe to hold stats data
df_stat = pd.DataFrame()

# Group by vertices and edges, and for each group dataframe:
groups = df_sorted.groupby(['Vertices', 'Edges'])
cum = 0
for name, group in groups:

    # Skip all groups with less than 30 elements (central limit theorem), or if group is a tree
    if len(group) < 30 or name[0] == name[1] + 1:
        print(f"Skipping group {name} because it has less than 30 graphs or it was a tree group")
        continue

    # Split into non-max and max graphs, and drop columns that we can't perform percentile calculations on
    non_maximum = group[group['Maximum'] == False].drop(columns=['G6id', 'Vertices', 'Edges', 'Vector', 'Degree', 'Maximum'])
    cum += len(non_maximum)
#     maximum     = group[group['Maximum'] == True ].drop(columns=['G6id', 'Vertices', 'Edges', 'Vector', 'Degree', 'Maximum'])
#
#     # Calculate percentile of score with strict mode, meaning only values less than the value is counted
#     for col in non_maximum.columns:
#         maximum[col] = stats.percentileofscore(non_maximum[col], maximum[col], kind='strict')
#
#     # Insert group name for first column
#     maximum.insert(0, 'Group', [name] * len(maximum))
#
#     # Append result to stat Dataframe
#     df_stat = pd.concat([df_stat, maximum])
#
# # Output to new csv
# df_stat.to_csv('dataset/stat.csv')
#
# #Display dataframe
# display(df_stat)

print(cum)

Skipping group (6, 5) because it has less than 30 graphs or it was a tree group
Skipping group (6, 6) because it has less than 30 graphs or it was a tree group
Skipping group (6, 7) because it has less than 30 graphs or it was a tree group
Skipping group (6, 8) because it has less than 30 graphs or it was a tree group
Skipping group (6, 9) because it has less than 30 graphs or it was a tree group
Skipping group (6, 10) because it has less than 30 graphs or it was a tree group
Skipping group (6, 11) because it has less than 30 graphs or it was a tree group
Skipping group (6, 12) because it has less than 30 graphs or it was a tree group
Skipping group (6, 13) because it has less than 30 graphs or it was a tree group
Skipping group (6, 14) because it has less than 30 graphs or it was a tree group
Skipping group (6, 15) because it has less than 30 graphs or it was a tree group
Skipping group (7, 6) because it has less than 30 graphs or it was a tree group
Skipping group (7, 16) because it 

In [23]:
# Graph visualization cell, don't run if you already have the images, also it takes about 30 seconds to run

# Set graph image size
plt.rcParams["figure.figsize"] = (5, 5)

# Create and save graph images of 3 different types of layout for each graph
for index, graph6id in zip(maximum_df.index, maximum_df['G6id']):

    # Create NetworkX graph instance from graph6id
    G = nx.from_graph6_bytes(graph6id.encode("ascii"))

    # Skip all tree graphs and all complete graphs
    if nx.is_tree(G) or G.size() == G.order() * (G.order() - 1) / 2:
        continue

    # Draw and save Kamada-Kawai layout
    nx.draw_kamada_kawai(G)
    plt.savefig(f"images/kamada_kawai/kamada_kawai_{index}.png")
    plt.clf()

    # Draw and save shell layout
    nx.draw_shell(G)
    plt.savefig(f"images/shell_layout/shell_{index}.png")
    plt.clf()

    # Draw and save spring layout
    nx.draw_spring(G)
    plt.savefig(f"images/spring_layout/spring_{index}.png")
    plt.clf()

<Figure size 500x500 with 0 Axes>